In [ ]:
!pip install plotly pandas requests ipywidgets

In [7]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import json
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
pyo.init_notebook_mode(connected=True)
def fetch_earthquake_data(start_date, end_date, min_magnitude=2.5):
    base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"    
    params = {
        'format': 'geojson',
        'starttime': start_date,
        'endtime': end_date,
        'minmagnitude': min_magnitude,
        'orderby': 'time'
    }
    try:
        print(f"Fetching earthquake data from {start_date} to {end_date}...")
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()        
        earthquakes = []
        for feature in data['features']:
            props = feature['properties']
            coords = feature['geometry']['coordinates']
            earthquake = {
                'id': feature['id'],
                'magnitude': props['mag'],
                'place': props['place'],
                'time': pd.to_datetime(props['time'], unit='ms'),
                'longitude': coords[0],
                'latitude': coords[1],
                'depth': coords[2],
                'alert': props.get('alert', 'green'),
                'tsunami': props.get('tsunami', 0),
                'sig': props.get('sig', 0),  
                'net': props.get('net', ''),  
                'type': props.get('type', 'earthquake'),
                'title': props.get('title', ''),
                'url': props.get('url', '')
            }
            earthquakes.append(earthquake)
        df = pd.DataFrame(earthquakes)
        print(f"Successfully fetched {len(df)} earthquake records.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()
def process_earthquake_data(df):
    if df.empty:
        return df
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour
    df['magnitude_category'] = pd.cut(df['magnitude'], bins=[0, 3, 4, 5, 6, 7, 10], 
                                    labels=['Minor', 'Light', 'Moderate', 'Strong', 'Major', 'Great'])
    df['depth_category'] = pd.cut(df['depth'], bins=[0, 35, 70, 300, 700], 
                                labels=['Shallow', 'Intermediate', 'Deep', 'Very Deep'])
    df['color'] = df['magnitude'].apply(lambda x: 
        'green' if x < 3 else
        'yellow' if x < 4 else
        'orange' if x < 5 else
        'red' if x < 6 else
        'darkred' if x < 7 else
        'purple')    
    df['size'] = df['magnitude'] * 3
    return df
def display_textual_analysis(df):
    if df.empty:
        print("No data available for analysis.")
        return
    print("\n" + "--------------------------")
    print("Comprehensive Earthquake Data Analysis:")
    print("--------------------------\n")    
    print(f"Basic Statistics:")
    print(f"   • Total Earthquakes: {len(df):,}")
    print(f"   • Date Range: {df['time'].min().strftime('%Y-%m-%d %H:%M')} to {df['time'].max().strftime('%Y-%m-%d %H:%M')}")
    print(f"   • Time Span: {(df['time'].max() - df['time'].min()).days} days")    
    print(f"\nMagnitude Analysis:")
    print(f"   • Range: {df['magnitude'].min():.1f} to {df['magnitude'].max():.1f}")
    print(f"   • Average: {df['magnitude'].mean():.2f}")
    print(f"   • Median: {df['magnitude'].median():.2f}")
    print(f"   • Standard Deviation: {df['magnitude'].std():.2f}")    
    strongest = df.nlargest(5, 'magnitude')[['magnitude', 'place', 'time']]
    print(f"\nTop 5 Strongest Earthquakes:")
    for idx, row in strongest.iterrows():
        print(f"   • {row['magnitude']:.1f} - {row['place']} ({row['time'].strftime('%Y-%m-%d %H:%M')})")
    print(f"\nDepth Analysis:")
    print(f"   • Range: {df['depth'].min():.1f} to {df['depth'].max():.1f} km")
    print(f"   • Average Depth: {df['depth'].mean():.1f} km")
    print(f"   • Median Depth: {df['depth'].median():.1f} km")
    print(f"\nMagnitude Categories:")
    category_counts = df['magnitude_category'].value_counts()
    for category, count in category_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   • {category}: {count:,} ({percentage:.1f}%)")
    print(f"\nDepth Categories:")
    depth_counts = df['depth_category'].value_counts()
    for category, count in depth_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   • {category}: {count:,} ({percentage:.1f}%)")
    print(f"\nGeographic Distribution:")
    print(f"   • Latitude Range: {df['latitude'].min():.2f}° to {df['latitude'].max():.2f}°")
    print(f"\nMost Active Regions:")
    regions = df['place'].str.extract(r'.*?,\s*(.*)')[0].value_counts().head(10)
    for region, count in regions.items():
        if pd.notna(region):
            percentage = (count / len(df)) * 100
            print(f"   • {region}: {count} earthquakes ({percentage:.1f}%)")    
    print(f"\nTemporal Patterns:")
    daily_avg = len(df) / (df['time'].max() - df['time'].min()).days
    print(f"   • Average per day: {daily_avg:.1f} earthquakes")
    most_active_days = df['date'].value_counts().head(5)
    print(f"   • Most Active Days:")
    for date, count in most_active_days.items():
        print(f"     - {date}: {count} earthquakes")
    hourly_dist = df['hour'].value_counts().sort_index()
    peak_hour = hourly_dist.idxmax()
    print(f"   • Peak Activity Hour: {peak_hour}:00 UTC ({hourly_dist[peak_hour]} earthquakes)")
    if 'alert' in df.columns:
        alert_counts = df['alert'].value_counts()
        if len(alert_counts) > 1:
            print(f"\nAlert Levels:")
            for alert, count in alert_counts.items():
                percentage = (count / len(df)) * 100
                print(f"   • {alert.title()}: {count:,} ({percentage:.1f}%)")
    if df['tsunami'].sum() > 0:
        tsunami_count = df['tsunami'].sum()
        print(f"\nTsunami Risk:")
        print(f"   • Earthquakes with tsunami potential: {tsunami_count}")
    print("\nAnalysis complete.")
print("System initialized and ready for earthquake analysis.")
date_start_widget = widgets.DatePicker(
    description='Start Date:',
    value=datetime.now().date() - timedelta(days=30),
    style={'description_width': 'initial'}
)
date_end_widget = widgets.DatePicker(
    description='End Date:',
    value=datetime.now().date(),
    style={'description_width': 'initial'}
)
magnitude_min_widget = widgets.FloatSlider(
    value=2.5,
    min=0.0,
    max=8.0,
    step=0.1,
    description='Min Magnitude:',
    style={'description_width': 'initial'},
    readout_format='.1f'
)
magnitude_max_widget = widgets.FloatSlider(
    value=10.0,
    min=2.0,
    max=10.0,
    step=0.1,
    description='Max Magnitude:',
    style={'description_width': 'initial'},
    readout_format='.1f'
)
input_container = widgets.VBox([
    widgets.HTML("<h3>Select Date Range:</h3>"),
    widgets.HBox([date_start_widget, date_end_widget]),
    widgets.HTML("<h3>Select Magnitude Range:</h3>"),
    widgets.HBox([magnitude_min_widget, magnitude_max_widget]),
])
display(input_container)
def create_earthquake_map(df, title="Global Earthquake Activity"):    
    if df.empty:
        print("No data to display.")
        return None
    df['hover_text'] = (
        '<b>' + df['title'] + '</b><br>' +
        'Magnitude: ' + df['magnitude'].astype(str) + '<br>' +
        'Depth: ' + df['depth'].round(2).astype(str) + ' km<br>' +
        'Time: ' + df['time'].dt.strftime('%Y-%m-%d %H:%M:%S') + '<br>' +
        'Location: ' + df['place'] + '<br>' +
        'Coordinates: (' + df['latitude'].round(3).astype(str) + ', ' + 
        df['longitude'].round(3).astype(str) + ')'
    )    
    fig = px.scatter_geo(df, 
                        lat='latitude', lon='longitude',
                        size='size', color='magnitude',
                        hover_name='title',
                        hover_data={'magnitude': True,'depth': ':.2f',
                        'time': '|%Y-%m-%d %H:%M:%S','place': True,
                        'size': False,'latitude': ':.3f',
                        'longitude': ':.3f'},
                        color_continuous_scale='Viridis',
                        size_max=30,
                        title=title)
    fig.update_layout(
        title_font_size=16,
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular'
        ),
        width=1000,
        height=600)
    return fig
def create_analysis_dashboard(df):
    if df.empty:
        print("No data available for visualization.")
        return
    print("Creating interactive earthquake map...")
    map_fig = create_earthquake_map(df, f"Earthquake Activity ({len(df)} events)")
    if map_fig:
        map_fig.show()
    print("Creating analysis dashboard...")
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Magnitude Distribution', 
            'Depth vs Magnitude', 
            'Daily Activity Timeline', 
            'Magnitude Categories'),
        specs=[
            [{"type": "xy"}, {"type": "xy"}],
            [{"type": "xy"}, {"type": "domain"}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    fig.add_trace(
        go.Histogram(x=df['magnitude'], nbinsx=20, name='Magnitude', 
                    marker=dict(color='lightblue', line=dict(color='darkblue', width=1))),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df['depth'], y=df['magnitude'], mode='markers',
                  marker=dict(size=8, color=df['magnitude'], colorscale='Viridis', 
                            line=dict(width=1, color='white')),
                  text=df['place'], name='Earthquakes'),
        row=1, col=2
    )
    daily_counts = df.groupby('date').size().reset_index(name='count')
    daily_counts['date'] = pd.to_datetime(daily_counts['date'])
    fig.add_trace(
        go.Scatter(x=daily_counts['date'], y=daily_counts['count'], 
                  mode='lines+markers', name='Daily Count',
                  line=dict(color='red', width=2), marker=dict(size=6)),
        row=2, col=1
    )
    category_counts = df['magnitude_category'].value_counts()
    fig.add_trace(
        go.Pie(labels=category_counts.index, values=category_counts.values,
               name="Categories"),
        row=2, col=2
    )
    fig.update_layout(
        height=800, 
        showlegend=False,
        title_text="Earthquake Analysis Dashboard",
        title_font_size=20
    )
    fig.update_xaxes(title_text="Magnitude", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_xaxes(title_text="Depth (km)", row=1, col=2)
    fig.update_yaxes(title_text="Magnitude", row=1, col=2)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Number of Earthquakes", row=2, col=1)
    fig.show()
def run_analysis():
    start_date = date_start_widget.value.strftime('%Y-%m-%d')
    end_date = date_end_widget.value.strftime('%Y-%m-%d')
    min_mag = magnitude_min_widget.value
    max_mag = magnitude_max_widget.value
    if date_start_widget.value >= date_end_widget.value:
        print("Error: Start date must be before end date.")
        return
    if min_mag >= max_mag:
        print("Error: Minimum magnitude must be less than maximum magnitude.")
        return
    print(f"\nStarting analysis with parameters:")
    print(f"   Date range: {start_date} to {end_date}")
    print(f"   Magnitude range: {min_mag} to {max_mag}")
    print("\n--------------------------\n") 
    df = fetch_earthquake_data(start_date, end_date, min_mag)
    if df.empty:
        print("No earthquake data found for the specified parameters.")
        print("Try adjusting your date range or lowering the minimum magnitude.")
        return
    df = process_earthquake_data(df)
    df = df[df['magnitude'] <= max_mag]
    if df.empty:
        print("No earthquakes found after applying magnitude filter.")
        return
    display_textual_analysis(df)
    print("\nCreating visualizations...")
    create_analysis_dashboard(df)
    print(f"\nAnalysis complete. Analyzed {len(df)} earthquakes.")
    return df
run_button = widgets.Button(
    description='Run Analysis',
    disabled=False,
    button_style='success',
    tooltip='Click to fetch data and create visualizations',
    icon='play',
    layout=widgets.Layout(width='200px', height='40px')
)
def on_run_button_clicked(b):
    with output:
        clear_output()
        global analysis_df
        analysis_df = run_analysis()
run_button.on_click(on_run_button_clicked)
output = widgets.Output()
display(widgets.VBox([
    widgets.HTML("<p>Ready to Analyze.<p>"),
    run_button,
    output
]))

System initialized and ready for earthquake analysis.
